In [1]:
# Import libraries and modules
import csv  # for reading and writing CSV files
import matplotlib.pyplot as plt  # for creating plots and charts
import numpy as np  # for numerical computing with Python
import pandas as pd  # for data manipulation and analysis
import pathlib  # for interacting with file paths in a cross-platform manner
import seaborn as sns  # for statistical data visualization
import tensorflow as tf  # for machine learning and deep learning
from keras.layers import Dense, Activation  # for building deep learning models in TensorFlow
from keras.models import Sequential  # for building deep learning models in TensorFlow
from keras.optimizers import Adam, RMSprop  # for building deep learning models in TensorFlow
from keras.callbacks import History
from sklearn.model_selection import train_test_split  # for model selection and evaluation
from sklearn.preprocessing import MinMaxScaler  # for preprocessing data
from tensorflow import keras  # for building deep learning models in TensorFlow
from tensorflow.keras import layers  # for building deep learning models in TensorFlow


# Print TensorFlow version
print(tf.__version__)

# Define column names for the CSV file
column_names = ['Open Time','Open','High','Low','Close',
                'Close Volume', 'Time', 'Quote asset volume',
               'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume']
               

# Read in the CSV file using pandas
raw_dataset = pd.read_csv("main.csv", names=column_names,
                      na_values = "?", comment='\t', dtype='float',
                      sep=",", skipinitialspace=True)

# Copy the data from the raw dataset to a new dataframe
dataset = raw_dataset.copy()

# Print the last few rows of the dataset
dataset.tail()


Matplotlib is building the font cache; this may take a moment.
2023-01-05 06:19:32.169698: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-05 06:19:39.740463: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-05 06:19:39.740618: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-05 06:19:39.740628: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Can

2.11.0


,Open Time,Open,High,Low,Close,Close Volume,Time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
188312,1.620794e+12,57517.42,57526.28,57485.00,57485.07,42.575735,1.620794e+12,2.448258e+06,1195.0,15.319691,880913.090845
188313,1.620794e+12,57485.07,57496.42,57466.75,57481.49,34.205467,1.620794e+12,1.966194e+06,1096.0,15.971891,918058.816162
188314,1.620794e+12,57477.18,57509.99,57458.18,57470.00,30.211789,1.620794e+12,1.736514e+06,955.0,13.054229,750364.577319
188315,1.620794e+12,57470.00,57470.01,57400.00,57450.90,45.354728,1.620794e+12,2.605080e+06,1559.0,12.615628,724559.233035
188316,1.620794e+12,57450.89,57475.66,57435.51,57450.19,14.168318,1.620794e+12,8.140594e+05,730.0,7.247751,416412.022206


In [2]:
dataset.pop("Open Time")
dataset.pop("Time")
# Randomly select 80% of the rows from the dataset and store them in a new dataframe
train_dataset = dataset.sample(frac=0.8,random_state=0)

# Remove the rows in the training dataset from the original dataset, leaving the remaining rows in a new dataframe
test_dataset = dataset.drop(train_dataset.index)


In [3]:
# Calculate statistical summary of the training dataset
train_stats = train_dataset.describe()

# Remove the "Close" column from the statistical summary
train_stats.pop("Close")

# Transpose the statistical summary so that it's in a more useful shape
train_stats = train_stats.transpose()

In [4]:
# Remove the "Close" column from the training dataset and store it in a new dataframe
train_labels = train_dataset.pop('Close')

# Remove the "Close" column from the testing dataset and store it in a new dataframe
test_labels = test_dataset.pop('Close')

In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_dataset)
normed_train_dataset = scaler.transform(train_dataset)
normed_test_dataset = scaler.transform(test_dataset)

In [6]:
print(normed_train_dataset.shape)
print(train_labels.shape)

(150654, 8)
(150654,)


In [7]:
print(train_labels)

54176     38273.80
22054     36653.35
25894     36570.67
3604      34482.99
96019     50880.06
            ...   
58124     46408.25
114876    57751.49
89163     51009.62
155465    56929.00
90475     48084.80
Name: Close, Length: 150654, dtype: float64


In [8]:
#=============== Parte Manual ===============#
neuronios = 32
act_h = "relu"
act_out = "sigmoid"
learnr = 0.001
camadas = 6
epochs_n = 30

# Set up neural network model
model = keras.Sequential([
    layers.Dense(neuronios, activation=act_h, input_shape=[8]),
    layers.Dense(neuronios, activation=act_h),
    layers.Dense(neuronios, activation=act_h),
    layers.Dense(neuronios, activation=act_h),
    layers.Dense(neuronios, activation=act_h),
    layers.Dense(neuronios, activation=act_h),
    layers.Dense(1,activation=act_h)
  ])

# Compile model
optimizer = Adam(learnr)

model.compile(loss='mse',optimizer=optimizer,metrics=['mae', 'mse'])

2023-01-05 06:19:47.080517: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 06:19:47.892883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 06:19:47.893951: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 06:19:47.896137: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 32)                1056      
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 32)                1056      
                                                                 
 dense_6 (Dense)             (None, 1)                 3

In [10]:
example_batch = normed_train_dataset[:8]
example_result = model.predict(example_batch)
example_result

1/1 [==============================] - 3s 3s/step


array([[0.02227697],
       [0.01879085],
       [0.01871964],
       [0.01471416],
       [0.04778258],
       [0.00899234],
       [0.04423985],
       [0.01642821]], dtype=float32)

In [11]:
# Set up callbacks
history = History()

# Train model
model.fit(normed_train_dataset, train_labels, verbose=2, batch_size=5, epochs= epochs_n, callbacks=[history])

Epoch 1/30
30131/30131 - 94s - loss: 12291802.0000 - mae: 369.5634 - mse: 12291802.0000 - 94s/epoch - 3ms/step
Epoch 2/30
30131/30131 - 87s - loss: 7471.4604 - mae: 65.0689 - mse: 7471.4604 - 87s/epoch - 3ms/step
Epoch 3/30
30131/30131 - 87s - loss: 7502.1836 - mae: 63.4426 - mse: 7502.1836 - 87s/epoch - 3ms/step
Epoch 4/30
30131/30131 - 85s - loss: 6897.1064 - mae: 61.8129 - mse: 6897.1064 - 85s/epoch - 3ms/step
Epoch 5/30
30131/30131 - 85s - loss: 6620.8584 - mae: 61.0726 - mse: 6620.8584 - 85s/epoch - 3ms/step
Epoch 6/30
30131/30131 - 89s - loss: 6599.9839 - mae: 61.0896 - mse: 6599.9839 - 89s/epoch - 3ms/step
Epoch 7/30
30131/30131 - 85s - loss: 6418.5166 - mae: 60.3757 - mse: 6418.5166 - 85s/epoch - 3ms/step
Epoch 8/30
30131/30131 - 86s - loss: 6205.6626 - mae: 59.3663 - mse: 6205.6626 - 86s/epoch - 3ms/step
Epoch 9/30
30131/30131 - 85s - loss: 6203.6113 - mae: 59.2727 - mse: 6203.6113 - 85s/epoch - 3ms/step
Epoch 10/30
30131/30131 - 88s - loss: 6092.5942 - mae: 58.9167 - mse: 609

In [12]:
# Evaluate model
results = model.evaluate(normed_test_dataset, test_labels)
print("Resultado",results)

# Print average loss over all epochs
print("Average loss:", np.mean(history.history['mse']))

1177/1177 [==============================] - 3s 2ms/step - loss: 5943.2900 - mae: 62.2333 - mse: 5943.2900
Resultado [5943.2900390625, 62.23331832885742, 5943.2900390625]
Average loss: 415401.82915039064


In [13]:
# predictions
y_pred = model.predict(normed_test_dataset)
print(y_pred)

1177/1177 [==============================] - 2s 2ms/step
[[28952.555]
 [28826.035]
 [28733.871]
 ...
 [57781.207]
 [57633.293]
 [57425.598]]


In [14]:
filename = 'modelmanual_c'+ str(camadas) +'_n'+ str(neuronios) +'_actn'+str(act_h) +'_acto'+str(act_out) +'_lr' + str(learnr) + '_lossmae' +'_ep' + str(epochs_n)
model.save(filename, save_format='tf')

INFO:tensorflow:Assets written to: modelmanual_c6_n32_actnrelu_actosigmoid_lr0.001_lossmae_ep30/assets
